# XPath e XQuery

# Base de dados e proposta
A base escolhida para essa análise é a : http://tbdb.bu.edu/tbdb_sysbio/Downloads.html

A TB database foi criada para auxiliar na pesquisa sobre a tuberculose. Ela contém informações referentes a diversos cromossomos relacionados à doença. Para esse trabalho, foram selecionados os dados de apenas um cromossomo (visto que a base era muito grande para trabalhar com ela inteira), e acreditamos que algumas pesquisas úteis poderiam ser realizadas com essas informações.

O primeiro passo para trabalhar com essa base foi arrumar os dados da base, que foram baixados no formato GTF (.gtf), para um arquivo XML (.xml). O Gene Transfer Format (GTF) é um formato de arquivo feito especialmente para guardar dados referentes a estrutura de genes. Para essa conversão, foi utilizado o seguinte código em C++ :

OBS: abrir a célula de markdown para conseguir ler o código.

#include <iostream>
#include <fstream>
#include <string>
#include <stdio.h>
#include<string.h>

using namespace std;

//componente para o vertex
struct GFT_LINE{
  string seqname;
  string type;
  string source;
  string feature;
  string start_value;
  string end_value;
  string score ;
  string strand;
  string frame;
  string gene_id;
  string transcript_id;
};

int main()
{
  string line;
  ifstream myfile ("rhodobacter_sphaeroides_2.4.1_1_transcripts.txt");
  ofstream outfile;
  outfile.open ("file.xml");
  outfile<<"<?xml version=\"1.0\"?>"<<endl;
  if (myfile.is_open()){
    GFT_LINE *beforeLine = new GFT_LINE;
    beforeLine->seqname = "";
    beforeLine->type = "";
    GFT_LINE *l;
    while ( getline (myfile,line) ){
      l = new GFT_LINE;
      string word;
      int start = 0;
      int part = 0;
      for(int i = 0; i<line.length(); i++){
        word = line.substr(start,i-start);
        if(word == "Chromosome_"){
          word = line.substr(start,i+1);
          l->seqname = word;
          l->type = line[i+2];
          i +=4;
          start = i;
        }else if(word == "gene_id "){
          start = i+1;
          for(int j= i+2; j < line.length(); j++){
            word = line.substr(start,j-start);
            if(line[j]=='\"'){
              i= j+2;
              start = j+2;
              break;
            }
          }
          i ++;
          start = i;
          l->gene_id = word;
        }else if(word == "transcript_id "){
          start = i+1;
          for(int j= i+2; j < line.length(); j++){
            word = line.substr(start,j-start);
            if(line[j]=='\"'){
              break;
            }
          }
          l->transcript_id = word;
          break;
        }else if(line[i] == '\t'){
           word = line.substr(start,i-start);
           start = i+1;
            if(part == 0){
              l->source = word;
            }else if(part == 1){
              l->feature = word;
            }else if(part == 2){
              l->start_value = word;
            }else if (part == 3){
              l->end_value = word;
            }else if (part == 4){
              l->score = word;
            }else if (part == 5){
              l->strand = word;
            }else if (part == 6){
              l->frame = word;
            }
            part++;
        }
      }
      int novo = 0;
      if(beforeLine->seqname != l->seqname){
        if(beforeLine->seqname != ""){
          outfile << "\t</type >" <<endl;
          outfile << "</" << beforeLine->seqname << ">" << endl;
        }
        outfile << "<" << l->seqname << ">" << endl;
      }
      if(beforeLine->type != l->type || beforeLine->seqname != l->seqname){
        if(beforeLine->seqname != ""){
          outfile << "\t</type >" <<endl;
        }
        outfile << "\t<type value=\""<< l->type << "\">" <<endl;
        novo = 1;
      }else
        novo = 0;
      outfile<<"\t\t<property " ;
      outfile<<"source=\""<< l->source<< "\" ";
      outfile<<"feature=\""<< l->feature<< "\" ";
      outfile<<"start=\""<< l->start_value<< "\" ";
      outfile<<"end=\""<< l->end_value<< "\" ";
      outfile<<"score=\""<< l->score<< "\" ";
      outfile<<"strand=\""<< l->strand<< "\" ";
      outfile<<"frame=\""<< l->frame<< "\" ";
      outfile<<"gene_id=\""<< l->gene_id<< "\" ";
      outfile<<"transcript_id=\""<< l->transcript_id<< "\" />" << endl;
      if((beforeLine->type != l->type && beforeLine->seqname == l->seqname) && !novo)
        outfile << "\t</type >" <<endl;
      beforeLine = l;
    }
    outfile << "\t</type >" <<endl;
    outfile << "</" << l->seqname << ">" << endl;
    myfile.close();
  }

  else cout << "Unable to open file";

    return 0;
}


Vale ressaltar que o código exige que o arquivo a ser convertido esteja no mesmo diretório que a main, e que sua extensão seja mudada para .txt antes da conversão.

O arquivo XML criado contém os seguintes dados:
    * source
    * feature
    * start
    * end
    * score
    * frame
    * attribute

## Proposta 1:
Uma primeira informação que pode ser interessante na pesquisa de um cromossomo é sua complexidade. Uma possível forma de se medir isso utilizando a base em XML seria contar o número de propriedades que um determinado tipo de um cromossomo possui.

```xquery
let $doc := doc("file2.xml")

return count($doc//type[@value = 1]/property)
```

**Resposta:** 3340

## Proposta 2:
Contar o número de códons de parada em um determinado tipo do cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 2])

return count($c/property[@feature = 'stop_codon'])
```

**Resposta:** 79

## Proposta 3:
Encontrar propriedades a determinado gene em um dado tipo de cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 4]/property)
where $c/@gene_id = 'RSP_2483'

return $c
```

**Algumas Respostas:** 
```xml
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="71408" end="71410" score="." strand="+" frame="0" gene_id="RSP_2483" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="72713" end="72715" score="." strand="+" frame="0" gene_id="RSP_2483" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="exon" start="71408" end="72715" score="." strand="+" frame="." gene_id="RSP_2483" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="CDS" start="71408" end="72712" score="." strand="+" frame="0" gene_id="RSP_2483" transcript_id="null"/>
```

## Proposta 4:
Encontrar propriedades de um tipo de cromossomo que comecem após uma determinada posição do cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 5]/property)
where $c/@start > 12000

return $c
```

**Algumas Respostas:** 
```xml
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="12384" end="12386" score="." strand="+" frame="0" gene_id="RSP_4150" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="12450" end="12452" score="." strand="+" frame="0" gene_id="RSP_4151" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="12768" end="12770" score="." strand="+" frame="0" gene_id="RSP_4151" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="exon" start="12450" end="12770" score="." strand="+" frame="." gene_id="RSP_4151" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="CDS" start="39087" end="39968" score="." strand="-" frame="0" gene_id="RSP_4189" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="40681" end="40683" score="." strand="+" frame="0" gene_id="RSP_4190" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="41272" end="41274" score="." strand="+" frame="0" gene_id="RSP_4190" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="exon" start="40681" end="41274" score="." strand="+" frame="." gene_id="RSP_4190" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="CDS" start="40681" end="41271" score="." strand="+" frame="0" gene_id="RSP_4190" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="41195" end="41197" score="." strand="+" frame="0" gene_id="RSP_4191" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="44135" end="44137" score="." strand="+" frame="0" gene_id="RSP_4191" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="exon" start="41195" end="44137" score="." strand="+" frame="." gene_id="RSP_4191" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="CDS" start="41195" end="44134" score="." strand="+" frame="0" gene_id="RSP_4191" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="44599" end="44601" score="." strand="+" frame="0" gene_id="RSP_4193" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="45151" end="45153" score="." strand="+" frame="0" gene_id="RSP_4193" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="exon" start="44599" end="45153" score="." strand="+" frame="." gene_id="RSP_4193" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="CDS" start="44599" end="45150" score="." strand="+" frame="0" gene_id="RSP_4193" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="45153" end="45155" score="." strand="+" frame="0" gene_id="RSP_4194" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="46185" end="46187" score="." strand="+" frame="0" gene_id="RSP_4194" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="exon" start="45153" end="46187" score="." strand="+" frame="." gene_id="RSP_4194" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="CDS" start="45153" end="46184" score="." strand="+" frame="0" gene_id="RSP_4194" transcript_id="null"/>
```

## Proposta 5:
Encontrar genes que remetem tanto a codons de início quanto a codons de parada em determinado tipo de cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 6]/property),
    $p in ($doc//type[@value = 6]/property)
where $c/@feature = 'start_codon' and
      $p/@feature = 'stop_codon' and
      $c/@gene_id = $p/@gene_id

return data($c/@gene_id)
```

**Algumas Respostas:** 
```xml
RSP_4254
RSP_4264
RSP_4265
RSP_4271
RSP_4278
RSP_4284
RSP_4285
RSP_4289
RSP_7352
RSP_7359
RSP_7360
RSP_7398
```

## Proposta 6:
Encontrar propriedades de um tipo de cromossomo que ocupem um "espaço" pequeno, ou seja, ocupam um trecho pequeno do cromossomo.

```xquery
let $doc := doc("file2.xml")

for $c in ($doc//type[@value = 1]/property)
where $c/@end - $c/@start <= 10

return $c
```

**Algumas Respostas:** 
```xml
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="926940" end="926942" score="." strand="-" frame="0" gene_id="RSP_1412" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="929652" end="929654" score="." strand="-" frame="0" gene_id="RSP_1413" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="930774" end="930776" score="." strand="+" frame="0" gene_id="RSP_1414" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="932860" end="932862" score="." strand="+" frame="0" gene_id="RSP_1415" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="932884" end="932886" score="." strand="+" frame="0" gene_id="RSP_1416" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="933170" end="933172" score="." strand="+" frame="0" gene_id="RSP_1417" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="934061" end="934063" score="." strand="+" frame="0" gene_id="RSP_1417" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="934330" end="934332" score="." strand="-" frame="0" gene_id="RSP_1418" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="936337" end="936339" score="." strand="-" frame="0" gene_id="RSP_1419" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="937278" end="937280" score="." strand="-" frame="0" gene_id="RSP_1420" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="936336" end="936338" score="." strand="-" frame="0" gene_id="RSP_1420" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="937519" end="937521" score="." strand="-" frame="0" gene_id="RSP_1421" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="939226" end="939228" score="." strand="-" frame="0" gene_id="RSP_1422" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="939206" end="939208" score="." strand="-" frame="0" gene_id="RSP_1423" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="941208" end="941210" score="." strand="+" frame="0" gene_id="RSP_1425" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="39397" end="39399" score="." strand="+" frame="0" gene_id="RSP_3003" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="39958" end="39960" score="." strand="+" frame="0" gene_id="RSP_3004" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="40564" end="40566" score="." strand="+" frame="0" gene_id="RSP_3004" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="41165" end="41167" score="." strand="-" frame="0" gene_id="RSP_3005" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="41397" end="41399" score="." strand="+" frame="0" gene_id="RSP_3006" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="42204" end="42206" score="." strand="-" frame="0" gene_id="RSP_3007" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="45328" end="45330" score="." strand="-" frame="0" gene_id="RSP_3012" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="44920" end="44922" score="." strand="-" frame="0" gene_id="RSP_3012" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="47046" end="47048" score="." strand="-" frame="0" gene_id="RSP_3014" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="47048" end="47050" score="." strand="-" frame="0" gene_id="RSP_3015" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="48887" end="48889" score="." strand="-" frame="0" gene_id="RSP_3016" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="49442" end="49444" score="." strand="-" frame="0" gene_id="RSP_3017" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="49518" end="49520" score="." strand="-" frame="0" gene_id="RSP_3018" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="51386" end="51388" score="." strand="-" frame="0" gene_id="RSP_3019" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="51853" end="51855" score="." strand="-" frame="0" gene_id="RSP_3020" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="52764" end="52766" score="." strand="-" frame="0" gene_id="RSP_3021" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="stop_codon" start="51852" end="51854" score="." strand="-" frame="0" gene_id="RSP_3021" transcript_id="null"/>
<property source="RSP1_INSERTASSEMBLYFROMGENBANK_1" feature="start_codon" start="53250" end="53252" score="." strand="+" frame="0" gene_id="RSP_3022" transcript_id="null"/>
```